In [7]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import feather
import re

from catboost import Pool, CatBoostRegressor,CatBoostClassifier
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import BayesianRidge

warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
train = feather.read_dataframe('best_table\\train')
test = feather.read_dataframe('best_table\\test')

In [4]:
target = pd.read_csv('target2',header=None,index_col=0)

In [4]:
target = (target < -25).astype(np.float64)

In [18]:
features = [c for c in train.columns if c not in ['card_id', 'first_active_month','last_active_month']]
categorical_feats = list(set([x for x in features if (re.search('modeplus',x) or re.search('feature_',x))] + [x for x in features if re.search('[th]_cat1_[^s]',x)]))

In [20]:
cat_feats = []
counter = 0
for x in train:
    if x in categorical_feats:
        cat_feats.append(counter)
    counter+=1

In [21]:
cat_feats

[49, 83, 99, 125, 128, 129, 143]

In [24]:
train_data = train
train_label = target
features = [c for c in train.columns if c not in ['card_id', 'first_active_month']]

# initialize Pool
train_pool = Pool(data=train.iloc[:40000,:][features], 
                  label=train_label[:40000], 
                  cat_features=cat_feats)
# test_pool = Pool(test_data[features], 
#                  cat_features=[0,1,2]) 

val_pool = Pool(data=train.iloc[40000:,:][features], 
                  label=train_label[40000:], 
                  cat_features=cat_feats)

CatboostError: Invalid type for cat_feature[3,125]=nan : cat_features must be integer or string, real number values and NaN values should be converted to string.

In [ ]:
model = CatBoostClassifier(iterations=4000, 
                        depth=10, 
                        learning_rate=0.007,
                        loss_function='Logloss',
                        task_type='GPU',
                        od_type='Iter',
                        use_best_model=2,
#                        l2_leaf_reg=500,      
#                        one_hot_max_size=10,
#                        random_strength=0.2,
#                        max_bin = 254,
                         )

model.fit(train_pool,eval_set=val_pool,use_best_model=True,early_stopping_rounds=100)

In [ ]:
test = feather.read_dataframe('test')

In [ ]:
test_pool = Pool(test[features], 
                 cat_features=[0,1,2])

In [ ]:
outliners = model.predict(test_pool)

In [ ]:
out_precent = model.predict_proba(test_pool)

In [ ]:
#feather.write_dataframe(pd.DataFrame(outliners.astype(np.bool_)),'test_outliner')

In [ ]:
test

In [ ]:
test[outliners]

In [ ]:
train = feather.read_dataframe('train')
#test = feather.read_dataframe('test')
target = pd.read_csv('target2',header=None,index_col=0)

In [ ]:
train['target'] = target
train = train[train['target'] > -20]
target = train['target']
del train['target']

In [ ]:
train.drop(labels=to_drop.index.values,axis=1,inplace=True)
test.drop(labels=to_drop.index.values,axis=1,inplace=True)

In [ ]:
train_data = train
train_label = target
#test_data = test

In [ ]:
features = [c for c in train.columns if c not in ['card_id', 'first_active_month']]

# initialize Pool
train_pool = Pool(data=train.iloc[:170000,:][features], 
                  label=train_label[:170000], 
                  cat_features=[0,1,2])

# test_pool = Pool(test_data[features], 
#                  cat_features=[0,1,2]) 

val_pool = Pool(data=train.iloc[170000:,:][features], 
                  label=train_label[170000:], 
                  cat_features=[0,1,2])

In [ ]:
model = CatBoostRegressor(iterations=6000, 
                          depth=12, 
                          learning_rate=0.05,
                          loss_function='RMSE',
                         task_type='GPU',
                          od_type='Iter',
                          use_best_model=2,
                          
                          l2_leaf_reg=500,
                          
                         one_hot_max_size=10,
#                         random_strength=0.5,
#                          max_bin = 254,
                          

                         )

model.fit(train_pool,eval_set=val_pool,use_best_model=True,early_stopping_rounds=200)


In [ ]:
model.drop_unused_features()

In [ ]:
model.get_feature_importance(prettified=True)

In [ ]:
test['target'] = 0

In [ ]:
testnon = test[np.invert(outliners.astype(np.bool_))]

In [ ]:
testout = test[outliners.astype(np.bool_)]

In [ ]:
test_pool = Pool(test[np.invert(outliners.astype(np.bool_))][features], 
                 cat_features=[0,1,2]) 

In [ ]:
testnon.loc[:,'target'] = model.predict(test_pool)

In [ ]:
testout.loc[:,'target'] = -33.219281

In [ ]:
test = testout.append(testnon)

In [ ]:
test = np.concatenate([testout,testnon],)

In [ ]:
test

In [ ]:
sub_df = pd.DataFrame({"card_id":test["card_id"].values,"target":test['target'].values})
sub_df.to_csv("submit.csv", index=False)